### Img, question, cross_attn_out are concatenated together
### Two norms
### Use Large_vocab_Epoch0002 weights
### vocab size 5500, stored in token_vocab
### whole model saved in VQA_final_model

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

import collections
import random
import numpy as np
import os
import time
import json
from PIL import Image
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

### Data Preparing

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !unzip drive/MyDrive/data.zip -d /content/data

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
question_file = "/content/drive/MyDrive/v2_OpenEnded_mscoco_train2014_questions.json"
question_file_val = "/content/drive/MyDrive/v2_OpenEnded_mscoco_val2014_questions.json"
IMG_PATH  = "/content/data/train2014/train2014/"

with open(question_file, "r") as f:
    question = json.load(f)

lemmatizer = WordNetLemmatizer()
question_image_pair = collections.defaultdict(list)
for item in question['questions']:
    question = item['question']
    tokens = word_tokenize(question)
    tokens = [t.lower() for t in tokens if t not in '''!,.;?()-[]{};:'"\<>/@#$+%^&*_~''']
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    question = " ".join(tokens)
    # image_path = IMG_PATH + 'COCO_train2014_' + '%012d.jpg' % (item['image_id'])
    image_path = IMG_PATH + 'COCO_train2014_' + '%012d.jpg' % (item['image_id'])
    question_image_pair[image_path].append(question)

with open(question_file_val, "r") as f:
    question = json.load(f)
IMG_PATH_VAL  = "/content/data/val2014/val2014/"
lemmatizer = WordNetLemmatizer()
question_image_pair_val = collections.defaultdict(list)
for item in question['questions']:
    question = item['question']
    tokens = word_tokenize(question)
    tokens = [t.lower() for t in tokens if t not in '''!,.;?()-[]{};:'"\<>/@#$+%^&*_~''']
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    question = " ".join(tokens)
    # image_path = IMG_PATH + 'COCO_train2014_' + '%012d.jpg' % (item['image_id'])
    image_path = IMG_PATH_VAL + 'COCO_val2014_' + '%012d.jpg' % (item['image_id'])
    question_image_pair_val[image_path].append(question)

In [ ]:
import random
answer_file = "/content/drive/MyDrive/v2_mscoco_train2014_annotations.json"
with open(answer_file, "r") as f:
    answer = json.load(f)
    
answer_image_pair = collections.defaultdict(list)
for item in answer['annotations']:
    ri = random.randint(0, 9)
    answer = f"{item['answers'][ri]['answer']}"
    tokens = word_tokenize(answer)
    tokens = [t.lower() for t in tokens if t not in '''!,.;?()-[]{};:'"\<>/@#$+%^&*_~''']
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    answer = " ".join(tokens)
    # image_path = IMG_PATH + 'COCO_train2014_' + '%012d.jpg' % (item['image_id'])
    image_path = IMG_PATH + 'COCO_train2014_' + '%012d.jpg' % (item['image_id'])
    answer_image_pair[image_path].append(answer)
    
answer_file_val = "/content/drive/MyDrive/v2_mscoco_val2014_annotations.json"
with open(answer_file_val, "r") as f:
    answer = json.load(f)
    
answer_image_pair_val = collections.defaultdict(list)
for item in answer['annotations']:
    ri = random.randint(0, 9)
    answer = f"{item['answers'][ri]['answer']}"
    tokens = word_tokenize(answer)
    tokens = [t.lower() for t in tokens if t not in '''!,.;?()-[]{};:'"\<>/@#$+%^&*_~''']
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    answer = " ".join(tokens)
    # image_path = IMG_PATH + 'COCO_train2014_' + '%012d.jpg' % (item['image_id'])
    image_path = IMG_PATH_VAL + 'COCO_val2014_' + '%012d.jpg' % (item['image_id'])
    answer_image_pair_val[image_path].append(answer)

In [ ]:
image_paths = list(answer_image_pair.keys())
random.shuffle(image_paths)
images_train = image_paths

image_paths_val = list(answer_image_pair_val.keys())
random.shuffle(image_paths_val)
images_val = image_paths_val

In [ ]:
question_train = []
answer_train = []
img_names =  []

for image_path in images_train:
    question_for_img = question_image_pair[image_path]
    question_train.extend(question_for_img)
    answer_for_img = answer_image_pair[image_path]
    answer_train.extend(answer_for_img)
    img_names.extend([image_path]*len(question_for_img))
print(len(question_train))
print(len(answer_train))
print(len(img_names))

443757
443757
443757


In [ ]:
question_val = []
answer_val = []
img_names_val =  []

for image_path in images_val:
    question_for_img = question_image_pair_val[image_path]
    question_val.extend(question_for_img)
    answer_for_img = answer_image_pair_val[image_path]
    answer_val.extend(answer_for_img)
    img_names_val.extend([image_path]*len(question_for_img))
print(len(question_val))
print(len(answer_val))
print(len(img_names_val))

214354
214354
214354


In [ ]:
from nltk.probability import FreqDist
fdist = FreqDist(answer_train)
most_5500 = fdist.most_common(5500)
most_5500 = [item[0] for item in most_5500]
most_5500 = [item for item in most_5500 if item not in ['', " "]]
# most_3000_l = [item[0] for item in most_3000 if len(item[0].split())>1]
# print(most_3000_l)
print(len(most_5500))

5499


In [ ]:
i = 0
for (question, answer, image) in zip(question_train, answer_train, img_names):
    if answer not in most_5500:
        question_train[i] = '<rev>'
        answer_train[i] = '<rev>'
        img_names[i] = '<rev>'
    i+=1

In [ ]:
i = 0
for (question, answer, image) in zip(question_val, answer_val, img_names_val):
    if  answer not in most_5500:
        question_val[i] = '<rev>'
        answer_val[i] = '<rev>'
        img_names_val[i] = '<rev>'
    i+=1

In [ ]:
question_train = [q for q in question_train if q != '<rev>']
answer_train = [q for q in answer_train if q != '<rev>']
img_names = [q for q in img_names if q != '<rev>']

question_val = [q for q in question_val if q != '<rev>']
answer_val = [q for q in answer_val if q != '<rev>']
img_names_val = [q for q in img_names_val if q != '<rev>']

In [ ]:
print(len(question_train))
print(len(answer_train))
print(len(img_names))

409769
409769
409769


In [ ]:
print(len(question_val))
print(len(answer_val))
print(len(img_names_val))

195346
195346
195346


In [ ]:
question_train_dataset = tf.data.Dataset.from_tensor_slices(question_train)
answer_train_dataset = tf.data.Dataset.from_tensor_slices(answer_train)
# mid_token_train_dataset = tf.data.Dataset.from_tensor_slices(mid_tokens_train)

question_val_dataset = tf.data.Dataset.from_tensor_slices(question_val)
answer_val_dataset = tf.data.Dataset.from_tensor_slices(answer_val)
# mid_token_val_dataset = tf.data.Dataset.from_tensor_slices(mid_tokens_val)


In [ ]:
print(most_5500[665])

on counter


In [ ]:
ans_tokenizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    standardize=None,
    ngrams = 3,
    output_sequence_length=1
)
ans_tokenizer.set_vocabulary('/content/drive/MyDrive/tokn_vocab')


In [ ]:
question_embedding = question_train_dataset.map(lambda x: x)
answer_embedding = answer_train_dataset.map(lambda x: ans_tokenizer(x))

question_embedding_val = question_val_dataset.map(lambda x: x)
answer_embedding_val = answer_val_dataset.map(lambda x: ans_tokenizer(x))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
len(ans_tokenizer.get_vocabulary())


5501

In [ ]:
word_to_index = tf.keras.layers.StringLookup(
    mask_token="",
    vocabulary=ans_tokenizer.get_vocabulary())
index_to_word = tf.keras.layers.StringLookup(
    mask_token="",
    vocabulary=ans_tokenizer.get_vocabulary(),
    invert=True)

In [ ]:
word_to_index('black')

<tf.Tensor: shape=(), dtype=int64, numpy=9>

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoImageProcessor, TFViTModel
import torch

def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.keras.layers.Resizing(224, 224)(img)
    img = img/255
    img = tf.transpose(img)
    return img

image_train_dataset = tf.data.Dataset.from_tensor_slices(img_names)
image_train_dataset = image_train_dataset.map(load_image)


image_val_dataset = tf.data.Dataset.from_tensor_slices(img_names_val)
image_val_dataset = image_val_dataset.map(load_image)


All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch32-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 128
train_dataset = tf.data.Dataset.zip((image_train_dataset, question_embedding))
train_dataset = tf.data.Dataset.zip((train_dataset, answer_embedding))
train_batches = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.zip((image_val_dataset, question_embedding_val))
val_dataset = tf.data.Dataset.zip((val_dataset, answer_embedding_val))
val_batches = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
pip install -q tf-models-official==2.11.0
pip install tensorflow_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 KB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### Vit Model

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch32-224-in21k")
vit_model = TFViTModel.from_pretrained("google/vit-base-patch32-224-in21k")
vit_model.trainable = True

### BERT Encoder

In [ ]:
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name='preprocessing')
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1", trainable=True, name='BERT_encoder')

### Training

In [ ]:
from model import QAModel
for (image, question), label in train_batches:
    break
sample_model = QAModel(vit_model=vit_model, bert_encoder=encoder)
sample_out = sample_model((image, question))
print(sample_out.shape)

(128, 1, 5695)


In [ ]:
epochs = 30
steps_per_epoch = tf.data.experimental.cardinality(train_batches).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "drive/MyDrive/Large_vocab_Epoch{epoch:04d}/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=int(steps_per_epoch))

# Create a new model instance
model = QAModel(vit_model=vit_model)
model.load_weights('/content/drive/MyDrive/Large_vocab_Epoch0002/cp-0002.ckpt')
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer=optimizer, metrics=['accuracy'])
# # Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_batches, epochs=30,
          callbacks=[cp_callback],
          validation_data=val_batches)

In [ ]:
model.evaluate(val_batches)

1527/1527 [==============================] - 835s 535ms/step - loss: 2.0407 - accuracy: 0.4458


[2.040680170059204, 0.44579872488975525]